# Notebook 03 - Data Cleaning

## Objectives
* Clean data
* Evaluate and process missing data

## Inputs
* outputs/datasets/collection/HousePricesRecords.csv

## Outputs
* Create Clean dataset:
    * all new datasets of cleaning will be stored in inputs/datasets/cleaning
* Split created dataset in to 2 parts:
    * Train
    * Test
* all new datasets (train and test) will be stored in outputs/datasets/cleaned

## Change working directory
In This section we will get location of current directory and move one step up, to parent folder, so App will be accessing project folder.

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [ ]:
import os

current_dir = os.getcwd()
current_dir

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chdir() defines the new current directory

In [ ]:
os.chdir(os.path.dirname(current_dir))
print("you have set a new current directory")

Confirm new current directory

In [ ]:
current_dir = os.getcwd()
current_dir

## Loading Dataset

In [ ]:
import pandas as pd

df = pd.read_csv("outputs/datasets/collection/HousePricesRecords.csv")
df.head()

Will make a copy of dataset, so later it can be compared with cleaned one

## Exploring Data

We will get all features that are missing data as a list

In [ ]:
features_with_missing_data = df.columns[df.isna().sum() > 0].to_list()
features_with_missing_data

### Visualizing Missing Data

Visualize the missing data to better understand the pattern of missing values.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Set the background style
sns.set(style="white")

# Prepare the data for heatmap, converting True/False to integers for color mapping
heatmap_data = df.isnull().astype(int)

# Visualize missing values as a heatmap
plt.figure(figsize=(10, 6))
# Using `cbar=True` to show the color bar
ax = sns.heatmap(heatmap_data, yticklabels=False, cbar=True, cmap='viridis',
                 cbar_kws={'label': 'Missing Data Indicator'})

# Customize the plot with titles and labels as needed
plt.title('Missing Data Heatmap', fontsize=16, color='navy')
plt.xticks(fontsize=12, color='darkred')  # Set x-tick colors to dark red
plt.yticks(fontsize=12, color='darkred')  # Set y-tick colors to dark red

# Set color bar label and style
cbar = ax.collections[0].colorbar
cbar.set_label('Missing Data Indicator', rotation=270, labelpad=20)
cbar.set_ticks([0.25, 0.75])  # Setting tick positions
cbar.set_ticklabels(['Present', 'Missing'])  # Setting tick labels

# Show the plot
plt.show()


We will generate a profiling report for features with missing data, which will assist us in selecting the most effective method for data cleaning.

In [ ]:
from ydata_profiling import ProfileReport

profile = ProfileReport(df=df[features_with_missing_data], minimal=True)
profile.to_notebook_iframe()

## All Data Cleaning

All Data cleaning with all steps, graphs, etc. will be performed in folder jupyter_notebooks/data_cleaning/

* All this is because we have noticed quite a few missing data vales, also we need to perform more deep checking on data is it all correct and valid.
* After All cleaning and fixing data in folder jupyter_notebooks/data_cleaning is completed, we will import inputs/datasets/cleaning/clean_finished.csv as current dataframe

When jupyter_notebooks/data_cleaning is being processed, we will add cleaning code below

In [ ]:
df.loc[:, 'LotFrontage'] = df['LotFrontage'].fillna(70)

# Lists of columns grouped by their fill values and type conversions
fill_zero_and_convert = ['1stFlrSF', '2ndFlrSF', 'GarageArea', 'GarageYrBlt',
                         'EnclosedPorch', 'MasVnrArea', 'WoodDeckSF', 'BedroomAbvGr']
fill_none = ['BsmtExposure', 'BsmtFinType1', 'GarageFinish']

# Fill missing values with zero and convert to integers for numerical columns
df[fill_zero_and_convert] = df[fill_zero_and_convert].fillna(0).astype(int)

# Fill missing values with 'None' for categorical columns
df[fill_none] = df[fill_none].fillna('None')
df['LotFrontage'] = df['LotFrontage'].round().astype(int)

df.loc[df['2ndFlrSF'] == 0, 'BedroomAbvGr'] = df['BedroomAbvGr'].replace(0, 2)
df.loc[df['2ndFlrSF'] > 0, 'BedroomAbvGr'] = df['BedroomAbvGr'].replace(0, 3)

# Swap values where '2ndFlrSF' is greater than '1stFlrSF'
swap_idx = df['2ndFlrSF'] > df['1stFlrSF']
df.loc[swap_idx, ['1stFlrSF', '2ndFlrSF']] = df.loc[swap_idx, ['2ndFlrSF', '1stFlrSF']].values

# Define features and their 'no presence' values
basement_features = ['BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtUnfSF', 'TotalBsmtSF']
features_and_values = {"BsmtExposure": "None", "BsmtFinType1": "None", "BsmtFinSF1": 0, "BsmtUnfSF": 0,
                       "TotalBsmtSF": 0}

# Check and update inconsistencies for each feature
for feature in basement_features:
    primary_value = features_and_values[feature]
    df['Consistency'] = df.apply(
        lambda row: all(row[f] == v for f, v in features_and_values.items()) if row[feature] == primary_value else True,
        axis=1
    )
    inconsistent_idx = df[~df['Consistency']].index
    if feature in ['BsmtExposure', 'BsmtFinType1']:
        correction = 'No' if feature == 'BsmtExposure' else 'Unf'
        df.loc[inconsistent_idx, feature] = correction

# Dropping new created column Consistency
df = df.drop(columns=['Consistency'])

# Correct zero values and adjust inconsistent records using vectorized operations
df.loc[df['BsmtUnfSF'] == 0, 'BsmtUnfSF'] = df['TotalBsmtSF'] - df['BsmtFinSF1']
df.loc[df['BsmtFinSF1'] == 0, 'BsmtFinSF1'] = df['TotalBsmtSF'] - df['BsmtUnfSF']
df.loc[df['TotalBsmtSF'] == 0, 'TotalBsmtSF'] = df['BsmtUnfSF'] + df['BsmtFinSF1']

# Identify and adjust records with inconsistent basement measurements using a ratio (example: 3)
mask = df['BsmtFinSF1'] + df['BsmtUnfSF'] != df['TotalBsmtSF']
df.loc[mask, 'BsmtUnfSF'] = (df.loc[mask, 'TotalBsmtSF'] / 3).astype(int)
df.loc[mask, 'BsmtFinSF1'] = df.loc[mask, 'TotalBsmtSF'] - df.loc[mask, 'BsmtUnfSF']

# Define a dictionary for checking consistency based on 'GarageFinish'
features_and_values = {"GarageArea": 0, "GarageFinish": 'None', "GarageYrBlt": 0}


def check_consistency(df, primary_feature):
    primary_value = features_and_values[primary_feature]
    return df.apply(
        lambda row: all(row[feature] == value for feature, value in features_and_values.items())
        if row[primary_feature] == primary_value else True, axis=1
    )


# Apply consistency check and correct 'GarageFinish'
consistency_mask = check_consistency(df, 'GarageFinish')
df.loc[~consistency_mask, 'GarageFinish'] = 'Unf'

# Correct garage years that are earlier than the house build year
df.loc[df['GarageYrBlt'] < df['YearBuilt'], 'GarageYrBlt'] = df['YearBuilt']

During cleaning, we did not drop any features, even some of them were missing nearly 90% of data.
We keep them to explore any potential correlations.

## Splitting data to train and test

In [ ]:
import os
from sklearn.model_selection import train_test_split

# Ensure output directories exist
output_dir = 'outputs/datasets/cleaned'
os.makedirs(output_dir, exist_ok=True)  # Creates the directory if it does not exist

# No need to separate features and target just yet, keep the dataframe whole for the split
df_train, df_test = train_test_split(df, test_size=0.2, random_state=0)

In [ ]:
# Save the datasets to PARQUET files including 'SalePrice'
df_train.to_parquet(f'{output_dir}/train.parquet.gzip', compression='gzip')
df_test.to_parquet(f'{output_dir}/test.parquet.gzip', compression='gzip')

## Correlation Study
### Initial study of correlations between features
Before we start checking correlations, we need to 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Compute the correlation matrix with numeric_only set to True
corr = df.corr(numeric_only=True)

# Create a heatmap
plt.figure(figsize=(15, 15))  # Set the size of the figure
plt.matshow(corr, cmap='coolwarm', fignum=1)  # Plot the correlation matrix as a heatmap

# Add color bar
plt.colorbar()

# Add labels to the x and y axes
plt.xticks(range(len(corr.columns)), corr.columns, rotation=45, ha='left')
plt.yticks(range(len(corr.columns)), corr.columns)

# Title for the heatmap
plt.title('Correlation Heatmap with Coefficients', pad=20)

# Add correlation coefficients on the heatmap
for (i, j), val in np.ndenumerate(corr.values):
    plt.text(j, i, f'{val:.2f}', ha='center', va='center', color='black')

# Show the plot
plt.show()


We will explore correlation (pearson and spearman methods) of Sales Price and all the rest features, sort values by absolute value and see most correlated features

* Before we process to correlation, all data objects needs to be encoded

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as ss
from sklearn.metrics import mutual_info_score


def plot_single_correlation(correlation, correlation_type, target):
    """
    Plots a single correlation type.
    correlation: DataFrame with correlation values
    correlation_type: String, name of the correlation type
    target: String, name of the target variable
    """
    plt.figure(figsize=(10, 6))
    ax = correlation.plot(kind='bar', color='lightgreen')
    plt.title(f'{correlation_type} Correlation Coefficients with {target}')
    plt.xlabel('Features')
    plt.ylabel(f'{correlation_type} Correlation Coefficient')
    plt.grid(True)

    for p in ax.patches:
        height = p.get_height()
        ax.annotate(f'{height:.2f}',
                    xy=(p.get_x() + p.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom')
    plt.show()


def calculate_numerical_correlations(df, target):
    """Calculates and returns a dictionary of correlation DataFrames for numerical features."""
    numerical_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    correlations = {}
    for method in ['Pearson', 'Spearman', 'Kendall']:
        corrs = []
        for col in numerical_cols:
            if col != target:
                if method == 'Pearson':
                    corr_value = ss.pearsonr(df[col], df[target])[0]
                elif method == 'Spearman':
                    corr_value = ss.spearmanr(df[col], df[target])[0]
                elif method == 'Kendall':
                    corr_value = ss.kendalltau(df[col], df[target])[0]
                corrs.append(corr_value)
        correlations[method] = pd.Series(corrs, index=[col for col in numerical_cols if col != target])
    return correlations


def calculate_categorical_correlations(df, target):
    """Calculates and returns a dictionary of correlation DataFrames for categorical features."""
    categorical_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()
    correlations = {'Cramer\'s V': {}, 'Mutual Information': {}}
    for col in categorical_cols:
        if col != target:
            table = pd.crosstab(df[col], df[target])
            chi2 = ss.chi2_contingency(table)[0]
            correlations['Cramer\'s V'][col] = np.sqrt(chi2 / (table.sum().sum() * (min(table.shape) - 1)))
            correlations['Mutual Information'][col] = mutual_info_score(df[col], df[target])
    for key in correlations:
        correlations[key] = pd.Series(correlations[key])
    return correlations


def plot_top_correlations(df, target, top_n=10):
    num_corr = calculate_numerical_correlations(df, target)
    cat_corr = calculate_categorical_correlations(df, target)
    all_correlations = {**num_corr, **cat_corr}

    # Plot individual correlation types
    for corr_type, corr_values in all_correlations.items():
        plot_single_correlation(corr_values, corr_type, target)

    # Convert dictionary of Series to DataFrame
    correlation_df = pd.DataFrame(all_correlations)

    # Calculate the maximum correlation value for each feature and determine the corresponding method
    max_values = correlation_df.max(axis=1)
    max_methods = correlation_df.idxmax(axis=1)

    # Create a DataFrame to store these maximum values along with their corresponding methods
    result_df = pd.DataFrame({
        'Feature': max_values.index,
        'Method': max_methods.values,
        'Value': max_values.values
    })

    # Sort the DataFrame by the 'Value' column in descending order to find the highest correlations
    result_df = result_df.sort_values(by='Value', ascending=False).head(top_n)

    # Plotting the results
    plt.figure(figsize=(12, 8))
    bars = plt.bar(result_df['Feature'] + " (" + result_df['Method'] + ")", result_df['Value'], color='skyblue')
    plt.title('Top Correlation Coefficients Across All Methods')
    plt.xlabel('Feature (Method)')
    plt.ylabel('Correlation Coefficient')
    plt.xticks(rotation=45, ha='right')
    plt.grid(True, which='both', linestyle='--', linewidth=0.5)

    # Annotating the bars with their correlation values
    for bar in bars:
        yval = bar.get_height()
        plt.text(bar.get_x() + bar.get_width() / 2, yval, f"{yval:.2f}", va='bottom')

    plt.tight_layout()
    plt.show()

    return result_df


list_of_top = plot_top_correlations(df_train, 'SalePrice', top_n=10)

We will print out our top 10 correlations:

In [ ]:
list_of_top

We have a list of features we will be investigating:
* BsmtFinType1
* KitchenQual
* OverallQual
* BsmtExposure
* GrLivArea
* GarageFinish
* 1stFlrSF
* TotalBsmtSF
* GarageArea
* YearBuilt

In [ ]:
# Converting to list all top features
top_correlations_list_hypothesis_1 = list_of_top.values.tolist()
# Making a list of features + SalePrice, so later it is easier to manage dataframe.
features_list = [item[0] for item in top_correlations_list_hypothesis_1]
features_list.append('SalePrice')

## EDA on selected features

We will create separate dataframe just with selected features + SalePrice

In [ ]:
df = df_train[features_list]
df

We will check how each feature is distributed against the Price, so we can see correlations in more details

In [ ]:
numeric_columns = df.columns

# Set the style
sns.set(style="whitegrid")

# Plot each feature against SalePrice
for column in numeric_columns[:-1]:
    sns.scatterplot(x=column, y='SalePrice', data=df)
    plt.title(f'{column} vs. SalePrice')
    plt.xlabel(column)
    plt.ylabel('SalePrice')
    plt.show()

## Outcomes

### Hypothesis - 1

Based on Plots above we can conclude these points:

* BsmtFinType1 - With Basement Quality increasing, house price tends to increase, although there is some high prices in Unfinished category
* KitchenQual - Kitchen Quality is very clear, with its quality going up, price also goes up.
* OverallQual - Overall Quality is most correlated feature from all, and when it increases, SalePrice also increases
* BsmtExposure - If There is basement in building, With better exposure price tends to increase
* GrLivArea - Increasing Ground Living Area SalePrice also increases
* GarageFinish - Just having garage, price increases, and the more it is Finished, the better house price
* 1stFlrSF - Similar to Ground Living Area, when it goes up, SalePrice also increases
* TotalBsmtSF - By increasing Basement we are increasing SalePrice
* GarageArea - When Garage Area goes up, Sale Price also increase. Most of the houses without garages are 150.000 or less
* YearBuilt - By increasing Garage Year Built we are increasing SalePrice, but it looks more exponential after 1980 ish...

## Next is Feature Engineering